In [ ]:
pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoConfig
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Use AutoModelForMaskedLM to load the model
model_name = "roberta-base"  # Change your model name here
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Use AutoTokenizer to load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use AutoConfig to load the config
config = AutoConfig.from_pretrained(model_name)

# Load your dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='small_reddit_left.csv',  # Replace with the path to your dataset after loading it
    block_size=128
)

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Fill training arguments
training_args = TrainingArguments(
    output_dir='roberta-pretrained-left',  # Change the output directory here
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    save_steps=10_000,
    save_total_limit=2,

    learning_rate=5e-5,
    fp16=True
)

# Create Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for 

Step,Training Loss
500,1.873900
1000,1.792700
1500,1.735900
2000,1.698500
2500,1.674200
3000,1.650100
3500,1.627700


TrainOutput(global_step=3837, training_loss=1.7123958696012918, metrics={'train_runtime': 1485.7179, 'train_samples_per_second': 82.629, 'train_steps_per_second': 2.583, 'total_flos': 8079814517651712.0, 'train_loss': 1.7123958696012918, 'epoch': 3.0})

In [ ]:
# Save it in .ckpt format
import torch
filename = f"roberta_pretrained_reddit_left.ckpt"
torch.save(model.state_dict(),filename)

In [ ]:
# Save it in a folder
model.save_pretrained('roberta-RL')
tokenizer.save_pretrained('roberta-RL')

('roberta-RL/tokenizer_config.json',
 'roberta-RL/special_tokens_map.json',
 'roberta-RL/vocab.json',
 'roberta-RL/merges.txt',
 'roberta-RL/added_tokens.json',
 'roberta-RL/tokenizer.json')